In [ ]:
import sqlite3, pandas as pd, numpy as np
from PIL import Image
import matplotlib.pyplot as plt 
from io import BytesIO 
import cv2 as cv

In [ ]:
con = sqlite3.connect("扫描图.SVD")
cur = con.cursor()
#cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
#cur.fetchall()
#ax = pd.read_sql_query("SELECT x_min, y_min FROM tile_raw_info;", con).plot.scatter(x='x_min', y='y_min') 

In [ ]:
cur.execute('''SELECT img0 FROM tile_raw_blob LIMIT 1;''') 
(img0,) = cur.fetchone()  
img_tile = np.asarray(bytearray(img0))  
img_tile = cv.imdecode(img_tile, cv.IMREAD_GRAYSCALE) 
h_tile, w_tile = img_tile.shape  
param = pd.read_sql_query("SELECT key, value FROM param;", con).to_dict('split')['data']
param = dict(param)  
h_cmbd, w_cmbd = int(param['height']), int(param['width'])
h_cmbd_new = (h_cmbd // h_tile + 1) * h_tile
w_cmbd_new = (w_cmbd // w_tile + 1) * w_tile
#print(h_tile, w_tile, h_cmbd_new, w_cmbd_new)


In [ ]:
#img_G = np.zeros((h_cmbd_new, w_cmbd_new)).astype(np.uint8)
#img_B = np.zeros((h_cmbd_new, w_cmbd_new)).astype(np.uint8) 
#img_R = np.zeros((h_cmbd_new, w_cmbd_new)).astype(np.uint8) 
img_G = np.ones((8200, 10000)).astype(np.uint8)
img_B = np.ones((8200, 10000)).astype(np.uint8) 
img_R = np.ones((8200, 10000)).astype(np.uint8) 
img_G[:] = 255
img_B[:] = 255
img_R[:] = 255
x = []
x_low_limit  = 14000
#x_high_limit = 18800
x_high_limit = 22800
y_low_limit  = 14000
y_high_limit = 20000
#y_high_limit = 18000

IMG_CMBD_COLLECTOR = []

for _, x_min, y_min, img0 in con.execute('''SELECT tile_raw_blob.id as blob_id, x_min, y_min, img0 
                     FROM tile_raw_info
                     JOIN tile_raw_blob ON tile_raw_info.blob_id = tile_raw_blob.id 
                     WHERE ch=0;'''): 
    if x_min < x_low_limit or x_min > x_high_limit or y_min < y_low_limit or y_min > y_high_limit: continue 
    x_min -= x_low_limit
    y_min -= y_low_limit
    x.append([x_min+w_tile, y_min+h_tile]) 
    img_tile = np.asarray(bytearray(img0))  
    img_tile = cv.imdecode(img_tile, cv.IMREAD_GRAYSCALE)   
    img_G[y_min:y_min+h_tile, x_min:x_min+w_tile] = img_tile 
    img_cmbd = np.stack((img_B, img_G, img_R),axis=2)
    imS = cv.resize(img_cmbd, (1600, 1600))[:,:,::-1] 
    IMG_CMBD_COLLECTOR.append(imS)

for _, x_min, y_min, img0 in con.execute('''SELECT tile_raw_blob.id as blob_id, x_min, y_min, img0 
                     FROM tile_raw_info
                     JOIN tile_raw_blob ON tile_raw_info.blob_id = tile_raw_blob.id 
                     WHERE ch=1;'''): 
    if x_min < x_low_limit or x_min > x_high_limit or y_min < y_low_limit or y_min > y_high_limit: continue
    x_min -=x_low_limit
    y_min -=y_low_limit
    img_tile = np.asarray(bytearray(img0))  
    img_tile = cv.imdecode(img_tile, cv.IMREAD_GRAYSCALE)  
    img_B[y_min:y_min+h_tile, x_min:x_min+w_tile] = img_tile
    img_cmbd = np.stack((img_B, img_G, img_R),axis=2)
    imS = cv.resize(img_cmbd, (1600, 1600))[:,:,::-1] 
    IMG_CMBD_COLLECTOR.append(imS)

for _, x_min, y_min, img0 in con.execute('''SELECT tile_raw_blob.id as blob_id, x_min, y_min, img0 
                     FROM tile_raw_info
                     JOIN tile_raw_blob ON tile_raw_info.blob_id = tile_raw_blob.id 
                     WHERE ch=2;'''): 
    if x_min < x_low_limit or x_min > x_high_limit or y_min < y_low_limit or y_min > y_high_limit: continue
    x_min -=x_low_limit
    y_min -=y_low_limit
    img_tile = np.asarray(bytearray(img0))  
    img_tile = cv.imdecode(img_tile, cv.IMREAD_GRAYSCALE)  
    img_R[y_min:y_min+h_tile, x_min:x_min+w_tile] = img_tile
    img_cmbd = np.stack((img_B, img_G, img_R),axis=2)
    imS = cv.resize(img_cmbd, (1600, 1600))[:,:,::-1] 
    IMG_CMBD_COLLECTOR.append(imS)

In [ ]:
import pickle
with open('a.pkl', 'wb') as fw:
    pickle.dump(IMG_CMBD_COLLECTOR, fw) 

In [1]:
import sqlite3, pandas as pd, numpy as np
from PIL import Image
import matplotlib.pyplot as plt 
from io import BytesIO 
import cv2 as cv
import pickle
with open('a.pkl', 'rb') as fr:
    IMG_CMBD_COLLECTOR = pickle.load(fr)

In [ ]:
%matplotlib inline 
plt.imshow(IMG_CMBD_COLLECTOR[89])
plt.show()

In [2]:
#pip install moviepy pygame  
#https://zulko.github.io/moviepy/getting_started/quick_presentation.html
from moviepy.editor import VideoClip, AudioFileClip, VideoFileClip 

def make_frame(_time): # run for every frame  
    return IMG_CMBD_COLLECTOR[int(_time)]
 
videoClip = VideoClip(make_frame, duration=len(IMG_CMBD_COLLECTOR)) 
#videoClip.ipython_display(fps=10, loop=True, autoplay=True)
videoClip.write_videofile(f'123.mp4', fps=30)  

Moviepy - Building video 123.mp4.
Moviepy - Writing video 123.mp4



Moviepy - Done !
Moviepy - video ready 123.mp4


In [ ]:
ffmpeg -i 123.mp4 -r 30 -filter:v "setpts=0.1*PTS" 123_10x.mp4